In [ ]:
# This notebook is divided into two main parts:

# Part 1: Model Validation
# In this section, the focus is on validating pre-trained models. 
# It involves assessing the performance of models that have already been trained, 
# using predefined parameters such as time index ranges for the validation dataset. 
# The goal is to evaluate the predictive accuracy of these models on test data.

# Part 2: Model Training
# This section is dedicated to training your own models. 
# It guides you through the process of setting up and configuring a new model, 
# including defining its parameters and architecture. 
# You will train the model using your dataset, allowing for experimentation 
# with different configurations to develop a model that best suits your data and prediction objectives.


In [1]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss,MAE,MAPE,RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
import pandas as pd

# Load the excel file
data = pd.read_excel('./dataset/hawaii.xlsx')
data["year"] = data["year"].astype(str)
data["day"] = data["day"].astype(str)
data['Holiday'] = data['Holiday'].astype(str)
data["tourist"] = data["tourist"].astype("float64")
data["Trend"]=data["Trend"].astype("float64")
data["Seasonal"]=data["Seasonal"].astype("float64")
data["Resid"]=data["Resid"].astype("float64")
data

,time_idx,date,tourist,destination,day of the week,month,year,day,Holiday,Trend,Seasonal,Resid
0,1,2009-07-01,22413.0,hawai,wednesday,July,2009,1,False,12964.379022,7002.230118,2446.390860
1,2,2009-07-02,23175.0,hawai,thursday,July,2009,2,False,12972.474815,7013.530694,3188.994492
2,3,2009-07-03,23756.0,hawai,friday,July,2009,3,Independence_day,12983.405389,7086.238553,3686.356058
3,4,2009-07-04,23443.0,hawai,saturday,July,2009,4,False,12998.073319,6645.230873,3799.695808
4,5,2009-07-05,21608.0,hawai,sunday,July,2009,5,False,13018.168501,4909.336516,3680.494983
...,...,...,...,...,...,...,...,...,...,...,...,...
4657,4658,2022-04-01,32302.0,hawai,friday,April,2022,1,False,19879.127737,7359.027611,5063.844652
4658,4659,2022-04-02,32510.0,hawai,saturday,April,2022,2,False,19878.984512,6599.167553,6031.847935
4659,4660,2022-04-03,30688.0,hawai,sunday,April,2022,3,False,19879.377208,7073.454562,3735.168230
4660,4661,2022-04-04,28972.0,hawai,monday,April,2022,4,False,19879.026226,6815.419236,2277.554537


In [9]:

# create dataloaders for model
batch_size = 1280  # set this between 32 to 128


# check if GPU is available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# move dataloaders to device



In [ ]:
# Part 1: Model Validation

In [10]:
# Trend

In [11]:
# Used for storing the trained model
pretrained_model_paths=['.\\saved_models\\best_model_epoch=28-v5.ckpt', '.\\saved_models\\best_model_epoch=25.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=27-v4.ckpt', '.\\saved_models\\best_model_epoch=25.ckpt', '.\\saved_models\\best_model_epoch=12.ckpt', '.\\saved_models\\best_model_epoch=28-v6.ckpt', '.\\saved_models\\best_model_epoch=25.ckpt', '.\\saved_models\\best_model_epoch=25.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=2-v6.ckpt', '.\\saved_models\\best_model_epoch=2-v6.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=28-v6.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=15.ckpt', '.\\saved_models\\best_model_epoch=2-v6.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=10-v1.ckpt', '.\\saved_models\\best_model_epoch=2-v6.ckpt', '.\\saved_models\\best_model_epoch=27-v4.ckpt', '.\\saved_models\\best_model_epoch=28-v6.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt', '.\\saved_models\\best_model_epoch=25.ckpt', '.\\saved_models\\best_model_epoch=28-v6.ckpt', '.\\saved_models\\best_model_epoch=28-v5.ckpt', '.\\saved_models\\best_model_epoch=28-v6.ckpt', '.\\saved_models\\best_model_epoch=28-v5.ckpt', '.\\saved_models\\best_model_epoch=27-v5.ckpt']

In [12]:
start_prediction_idx = 4573
end_prediction_idx = 4662  # Includes 90 time points
prediction_length = 3  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []



In [13]:
# Used for creating the validation set
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 4481], 
        time_idx="time_idx",
        target="Trend",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Trend"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


In [14]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

# Print all predictions
print(all_predictions)

[13597.08203125, 13499.8583984375, 13580.2158203125, 12888.91015625, 12529.318359375, 12034.083984375, 12998.0986328125, 12727.958984375, 13180.39453125, 10877.1826171875, 11031.2646484375, 10146.0634765625, 9473.966796875, 9240.103515625, 9024.0869140625, 9038.689453125, 9171.265625, 9552.646484375, 8826.5, 8931.0830078125, 9135.412109375, 8144.7880859375, 8481.3017578125, 8176.919921875, 8381.3212890625, 8415.6015625, 8865.787109375, 9985.154296875, 9996.673828125, 10411.7275390625, 11433.7373046875, 10882.3349609375, 11012.2265625, 12866.8603515625, 13197.11328125, 12637.21484375, 13810.5029296875, 13344.2099609375, 12073.49609375, 16258.83984375, 16412.767578125, 16809.52734375, 18082.080078125, 18133.8984375, 18218.751953125, 18233.99609375, 18431.109375, 18012.072265625, 14914.078125, 14644.2744140625, 14337.1376953125, 12928.205078125, 13440.83203125, 12699.7939453125, 15289.9482421875, 15424.8525390625, 15604.87890625, 16293.47265625, 15172.166015625, 15581.2265625, 13786.77734

In [15]:
Trend_fore=[13597.08203125, 13499.8583984375, 13580.2158203125, 12888.91015625, 12529.318359375, 12034.083984375, 12998.0986328125, 12727.958984375, 13180.39453125, 10877.1826171875, 11031.2646484375, 10146.0634765625, 9473.966796875, 9240.103515625, 9024.0869140625, 9038.689453125, 9171.265625, 9552.646484375, 8826.5, 8931.0830078125, 9135.412109375, 8144.7880859375, 8481.3017578125, 8176.919921875, 8381.3212890625, 8415.6015625, 8865.787109375, 9985.154296875, 9996.673828125, 10411.7275390625, 11433.7373046875, 10882.3349609375, 11012.2265625, 12866.8603515625, 13197.11328125, 12637.21484375, 13810.5029296875, 13344.2099609375, 12073.49609375, 16258.83984375, 16412.767578125, 16809.52734375, 18082.080078125, 18133.8984375, 18218.751953125, 18233.99609375, 18431.109375, 18012.072265625, 14914.078125, 14644.2744140625, 14337.1376953125, 12928.205078125, 13440.83203125, 12699.7939453125, 15289.9482421875, 15424.8525390625, 15604.87890625, 16293.47265625, 15172.166015625, 15581.2265625, 13786.77734375, 13320.6796875, 13757.53515625, 16993.392578125, 17199.419921875, 17383.7734375, 17842.10546875, 18146.037109375, 17896.169921875, 19627.181640625, 19730.193359375, 19926.486328125, 20701.66796875, 21043.30859375, 21747.611328125, 20074.0859375, 19831.72265625, 19246.78515625, 21434.62890625, 21176.943359375, 21368.021484375, 19445.205078125, 19147.40625, 18831.5859375, 20738.77734375, 20872.173828125, 20613.966796875, 20203.755859375, 20233.466796875, 19958.509765625]

In [16]:
# Seasonal

In [17]:
pretrained_model_paths=['.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v11.ckpt', '.\\saved_models\\best_model_epoch=3-v12.ckpt', '.\\saved_models\\best_model_epoch=3-v11.ckpt', '.\\saved_models\\best_model_epoch=27-v6.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=6-v5.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=8-v3.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=2-v8.ckpt', '.\\saved_models\\best_model_epoch=8.ckpt', '.\\saved_models\\best_model_epoch=3.ckpt', '.\\saved_models\\best_model_epoch=6.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=20-v2.ckpt', '.\\saved_models\\best_model_epoch=4.ckpt', '.\\saved_models\\best_model_epoch=3-v2.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt', '.\\saved_models\\best_model_epoch=3-v10.ckpt']

In [18]:
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 4481], 
        time_idx="time_idx",
        target="Seasonal",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Seasonal"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


In [19]:
all_predictions = []


for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3] 
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

print(all_predictions)

[6862.55908203125, 6718.25146484375, 6435.71826171875, 6859.42822265625, 6672.87451171875, 6682.455078125, 6951.55859375, 6971.3642578125, 7006.7548828125, 6754.29736328125, 7121.6640625, 6782.5869140625, 6610.5791015625, 6841.017578125, 6745.29345703125, 7511.2724609375, 7213.71484375, 7062.18408203125, 6051.58154296875, 6296.08544921875, 5897.037109375, 7323.853515625, 7248.537109375, 7149.60546875, 6969.5859375, 6939.41162109375, 7026.7802734375, 6973.90966796875, 7061.0341796875, 7045.24462890625, 6923.0126953125, 7106.99072265625, 6986.59912109375, 6539.60546875, 6931.1435546875, 6753.904296875, 7010.74169921875, 7085.79931640625, 7035.240234375, 6504.943359375, 6901.7890625, 6961.4208984375, 6957.5107421875, 7023.43310546875, 6902.2841796875, 7027.48583984375, 7015.9921875, 6981.97802734375, 6732.6533203125, 6603.4296875, 6783.4580078125, 7074.35400390625, 7053.39013671875, 7042.44140625, 7043.923828125, 6977.63818359375, 7067.14697265625, 7015.0732421875, 7044.86181640625, 7029.

In [20]:
Seasonal_fore=[6862.55908203125, 6718.25146484375, 6435.71826171875, 6859.42822265625, 6672.87451171875, 6682.455078125, 6951.55859375, 6971.3642578125, 7006.7548828125, 6754.29736328125, 7121.6640625, 6782.5869140625, 6610.5791015625, 6841.017578125, 6745.29345703125, 7511.2724609375, 7213.71484375, 7062.18408203125, 6051.58154296875, 6296.08544921875, 5897.037109375, 7323.853515625, 7248.537109375, 7149.60546875, 6969.5859375, 6939.41162109375, 7026.7802734375, 6973.90966796875, 7061.0341796875, 7045.24462890625, 6923.0126953125, 7106.99072265625, 6986.59912109375, 6539.60546875, 6931.1435546875, 6753.904296875, 7010.74169921875, 7085.79931640625, 7035.240234375, 6504.943359375, 6901.7890625, 6961.4208984375, 6957.5107421875, 7023.43310546875, 6902.2841796875, 7027.48583984375, 7015.9921875, 6981.97802734375, 6732.6533203125, 6603.4296875, 6783.4580078125, 7074.35400390625, 7053.39013671875, 7042.44140625, 7043.923828125, 6977.63818359375, 7067.14697265625, 7015.0732421875, 7044.86181640625, 7029.7216796875, 7054.31689453125, 7059.517578125, 7034.43115234375, 7051.19287109375, 7026.990234375, 7021.03076171875, 7045.4169921875, 7087.9814453125, 7074.732421875, 7013.35888671875, 7077.75732421875, 7072.34130859375, 7114.193359375, 7250.59521484375, 7014.7021484375, 7147.455078125, 7111.30029296875, 7180.82373046875, 6828.85009765625, 6541.9755859375, 6660.3115234375, 6537.53271484375, 6579.640625, 6865.15283203125, 6640.94921875, 6981.0703125, 6570.10107421875, 6947.6611328125, 6610.1435546875, 6771.71484375]

In [21]:
# Resid

In [22]:
pretrained_model_paths=['.\\saved_models\\best_model_epoch=26-v6.ckpt', '.\\saved_models\\best_model_epoch=26-v7.ckpt', '.\\saved_models\\best_model_epoch=20-v3.ckpt', '.\\saved_models\\best_model_epoch=28-v8.ckpt', '.\\saved_models\\best_model_epoch=28-v9.ckpt', '.\\saved_models\\best_model_epoch=29-v5.ckpt', '.\\saved_models\\best_model_epoch=24-v5.ckpt', '.\\saved_models\\best_model_epoch=26-v8.ckpt', '.\\saved_models\\best_model_epoch=28-v10.ckpt', '.\\saved_models\\best_model_epoch=26-v9.ckpt', '.\\saved_models\\best_model_epoch=28-v11.ckpt', '.\\saved_models\\best_model_epoch=27-v7.ckpt', '.\\saved_models\\best_model_epoch=28-v12.ckpt', '.\\saved_models\\best_model_epoch=26-v10.ckpt', '.\\saved_models\\best_model_epoch=28-v13.ckpt', '.\\saved_models\\best_model_epoch=12-v3.ckpt', '.\\saved_models\\best_model_epoch=20-v4.ckpt', '.\\saved_models\\best_model_epoch=10-v4.ckpt', '.\\saved_models\\best_model_epoch=23.ckpt', '.\\saved_models\\best_model_epoch=5-v12.ckpt', '.\\saved_models\\best_model_epoch=22-v3.ckpt', '.\\saved_models\\best_model_epoch=31-v2.ckpt', '.\\saved_models\\best_model_epoch=14-v2.ckpt', '.\\saved_models\\best_model_epoch=24-v6.ckpt', '.\\saved_models\\best_model_epoch=34-v2.ckpt', '.\\saved_models\\best_model_epoch=22-v4.ckpt', '.\\saved_models\\best_model_epoch=6-v7.ckpt', '.\\saved_models\\best_model_epoch=34-v3.ckpt', '.\\saved_models\\best_model_epoch=34-v4.ckpt', '.\\saved_models\\best_model_epoch=30.ckpt']

In [23]:
training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= 4481], 
        time_idx="time_idx",
        target="Resid",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Resid"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


In [24]:
all_predictions = []


for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1 
    
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

print(all_predictions)

[3483.036865234375, 3582.214599609375, 5498.6552734375, 3401.073486328125, 2136.7568359375, 787.735107421875, 2240.84912109375, 3518.04833984375, 3574.552490234375, 5470.0625, 3191.804931640625, 2165.94091796875, 575.80517578125, 1975.2862548828125, 3513.6220703125, 3905.203125, 5597.3369140625, 3158.726318359375, 2066.937255859375, 1177.3740234375, 2333.4990234375, 3564.8330078125, 3981.0634765625, 5820.123046875, 3111.43408203125, 2301.912109375, 996.6947021484375, 2278.860107421875, 3787.37060546875, 3995.360595703125, 5388.1953125, 3232.330322265625, 2299.44189453125, 878.255126953125, 1517.4710693359375, 3380.29150390625, 3767.2197265625, 5811.90185546875, 3191.114013671875, 2055.29736328125, 489.356689453125, 1876.90869140625, 4027.253662109375, 3924.711669921875, 5981.8544921875, 3078.496337890625, 2276.95361328125, 1010.115966796875, 1662.45654296875, 2832.6630859375, 3276.50048828125, 2829.02880859375, 3174.861328125, 2969.73828125, 914.26318359375, 1600.1431884765625, 3546.96

In [25]:
Resid_fore=[3483.036865234375, 3582.214599609375, 5498.6552734375, 3401.073486328125, 2136.7568359375, 787.735107421875, 2240.84912109375, 3518.04833984375, 3574.552490234375, 5470.0625, 3191.804931640625, 2165.94091796875, 575.80517578125, 1975.2862548828125, 3513.6220703125, 3905.203125, 5597.3369140625, 3158.726318359375, 2066.937255859375, 1177.3740234375, 2333.4990234375, 3564.8330078125, 3981.0634765625, 5820.123046875, 3111.43408203125, 2301.912109375, 996.6947021484375, 2278.860107421875, 3787.37060546875, 3995.360595703125, 5388.1953125, 3232.330322265625, 2299.44189453125, 878.255126953125, 1517.4710693359375, 3380.29150390625, 3767.2197265625, 5811.90185546875, 3191.114013671875, 2055.29736328125, 489.356689453125, 1876.90869140625, 4027.253662109375, 3924.711669921875, 5981.8544921875, 3078.496337890625, 2276.95361328125, 1010.115966796875, 1662.45654296875, 2832.6630859375, 3276.50048828125, 2829.02880859375, 3174.861328125, 2969.73828125, 914.26318359375, 1600.1431884765625, 3546.962890625, 2895.609375, 3126.922607421875, 2999.67431640625, 2090.041259765625, 4.777979850769043, 1775.0262451171875, 4216.32666015625, 4189.85302734375, 6108.05859375, 3380.0146484375, 1872.064453125, 693.504150390625, 2171.658447265625, 3601.46240234375, 4051.18017578125, 5937.77294921875, 3725.7734375, 2284.584228515625, 630.819580078125, 1834.722412109375, 3346.911865234375, 3119.81982421875, 3155.253173828125, 3107.82421875, 1859.14892578125, 323.94189453125, 1627.3291015625, 3594.16650390625, 4013.964111328125, 6142.6728515625, 3322.86767578125, 2200.896484375, 443.3623046875]

In [26]:
true_values=[24367.0, 23420.0, 26130.0, 23439.0, 19610.0, 18586.0, 21759.0, 24375.0, 24357.0, 25707.0, 20888.0, 17778.0, 16533.0, 17146.0, 19909.0, 20142.0, 22404.0, 19821.0, 16885.0, 15689.0, 17740.0, 19602.0, 19686.0, 21146.0, 17977.0, 16667.0, 20992.0, 20126.0, 20983.0, 21053.0, 23918.0, 21208.0, 18788.0, 19243.0, 21748.0, 24712.0, 24800.0, 27108.0, 22346.0, 20723.0, 25169.0, 26711.0, 28185.0, 29331.0, 30886.0, 29440.0, 27390.0, 25170.0, 23266.0, 24112.0, 24602.0, 27227.0, 24024.0, 22834.0, 24811.0, 23876.0, 26507.0, 27046.0, 28263.0, 25053.0, 22183.0, 21253.0, 25355.0, 28039.0, 30709.0, 30874.0, 29531.0, 27357.0, 24511.0, 26647.0, 30572.0, 31853.0, 33923.0, 31957.0, 30821.0, 28001.0, 28761.0, 29740.0, 31250.0, 33341.0, 29855.0, 28841.0, 25083.0, 27158.0, 30886.0, 32302.0, 32510.0, 30688.0, 28972.0, 27104.0]

In [27]:
import numpy as np
# Combine trend, season, and residual components to get the final 'tourist' forecast

tourist = [sum(x) for x in zip(Trend_fore, Seasonal_fore, Resid_fore)]



# Calculate Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and Mean Absolute Percentage Error (MAPE)
mae = np.mean(np.abs(np.array(true_values) - np.array(tourist)))
rmse = np.sqrt(np.mean(np.square(np.array(true_values) - np.array(tourist))))
mape = np.mean(np.abs(np.array(true_values) - np.array(tourist)) / np.array(true_values))

# Print the evaluation metrics
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

MAE: 789.4766056166754
RMSE: 1213.7113898309917
MAPE: 0.03289312817427155


In [ ]:
# Part 2: Model Training

In [8]:
# trend

# Define the time index range for the validation dataset
start_prediction_idx = 4573 # Starting index for prediction
end_prediction_idx = 4662 # Ending index for prediction


# Set the prediction length to 3, indicating that the model predicts 3 steps ahead
prediction_length = 3

# Note: The test set comprises 90 time points. The prediction_length of 3 means that 
# the models trained in this session are designed to forecast 3 steps ahead. The 
# start and end indices (4573 to 4662) specify the time points covered by the 
# model's predictions. These indices also determine the number of models to be trained, 
# calculated as: (end_prediction_idx - start_prediction_idx + 1) / prediction_length.
# For start_prediction_idx = 4573 and end_prediction_idx = 4662 with prediction_length = 3,
# the total number of models trained in the loop is 30.

max_encoder_length = 30 

# Initialize an empty list to store all prediction values
all_predictions = []

# Initialize an empty list to store the best models for each prediction
all_best_models = []

In [ ]:
# Caution: The models are resource-intensive, requiring a high-performance GPU. Training 
# too many models in a single loop may lead to exhaustion of computer resources, causing 
# the program to crash. In such cases, you may need to restart the program or adjust the 
# start and end indices to reduce the number of models trained in the loop.

In [ ]:



for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
   
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="Trend",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Trend"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)

    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    


    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  
    logger = TensorBoardLogger("lightning_logs")  

    trainer = pl.Trainer(
        max_epochs=30,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.03911626926390909,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.082239225958163498,
        hidden_size=125,
        attention_head_size=3,
        dropout=0.15160823136480017,
        hidden_continuous_size=17,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [9]:
all_best_models

In [ ]:
# seasonal

In [ ]:
start_prediction_idx = 4573
end_prediction_idx = 4662 

prediction_length = 3


max_encoder_length = 30 

all_predictions = []
all_best_models = []

In [ ]:


for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  # 更新训练集的时间索引上限
        time_idx="time_idx",
        target="Seasonal",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Seasonal"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)

    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # configure network and trainer
    

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=30,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.03911626926390909,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.022239225958163498,
        hidden_size=125,
        attention_head_size=3,
        dropout=0.15160823136480017,
        hidden_continuous_size=17,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    

    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )


    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 


In [ ]:
all_best_models

In [ ]:
# Resid

In [ ]:

start_prediction_idx = 4573
end_prediction_idx = 4662

prediction_length = 3


max_encoder_length = 30 
all_predictions = []
all_best_models = []

In [ ]:


for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="Resid",
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month", "day of the week", "day", "Holiday"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["Resid"],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )


    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)

    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # configure network and trainer
    

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1, 
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=30,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.03911626926390909,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.022239225958163498,
        hidden_size=125,
        attention_head_size=3,
        dropout=0.15160823136480017,
        hidden_continuous_size=17,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    

    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )


    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 


In [ ]:
all_best_models